In [23]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import csv
from IPython.display import clear_output

In [7]:
from sklearn.model_selection import train_test_split

In [4]:
def generated_images(image_folder,color):
  images,images_name = [],[]
  i = 0
  for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
      image_path = os.path.join(image_folder, filename)
      image = Image.open(image_path).convert(color).resize((256,256))  # Convert to RGB
      image_array = np.array(image).flatten()
      images.append(image_array)
      images_name.append(filename)
      print(f"Image no. {i}")
      clear_output(wait=True)
      i += 1
  return images,images_name

In [ ]:
image_folder = r'soil_classification-2025/train'
images,images_name = generated_images(image_folder,'L')
with open('images.csv', 'w', newline='') as csvfile:
	writer = csv.writer(csvfile)
	writer.writerow(['Image Name'] + [f'Pixel {i}' for i in range(256*256)])  # Header
	for name, image in zip(images_name, images):
		writer.writerow([name] + list(image))

Image no. 1213


In [8]:
images_df = pd.read_csv('images_grey.csv')

In [9]:
labels = pd.read_csv(r'soil_classification-2025\train_labels.csv')
labels.rename(columns={'image_id': 'Image Name'}, inplace=True)

In [10]:
images_df = images_df.merge(labels, on='Image Name', how='left')


In [11]:
images_df.drop(columns=['Image Name'],inplace = True)
# images_df.rename(columns={'soil_type_x': 'soil_type'}, inplace=True)
images_df

,Pixel 0,Pixel 1,Pixel 2,Pixel 3,Pixel 4,Pixel 5,Pixel 6,Pixel 7,Pixel 8,Pixel 9,Pixel 10,Pixel 11,Pixel 12,Pixel 13,Pixel 14,Pixel 15,Pixel 16,Pixel 17,Pixel 18,Pixel 19,Pixel 20,Pixel 21,Pixel 22,Pixel 23,Pixel 24,Pixel 25,Pixel 26,Pixel 27,Pixel 28,Pixel 29,Pixel 30,Pixel 31,Pixel 32,Pixel 33,Pixel 34,Pixel 35,Pixel 36,Pixel 37,Pixel 38,Pixel 39,...,Pixel 65497,Pixel 65498,Pixel 65499,Pixel 65500,Pixel 65501,Pixel 65502,Pixel 65503,Pixel 65504,Pixel 65505,Pixel 65506,Pixel 65507,Pixel 65508,Pixel 65509,Pixel 65510,Pixel 65511,Pixel 65512,Pixel 65513,Pixel 65514,Pixel 65515,Pixel 65516,Pixel 65517,Pixel 65518,Pixel 65519,Pixel 65520,Pixel 65521,Pixel 65522,Pixel 65523,Pixel 65524,Pixel 65525,Pixel 65526,Pixel 65527,Pixel 65528,Pixel 65529,Pixel 65530,Pixel 65531,Pixel 65532,Pixel 65533,Pixel 65534,Pixel 65535,soil_type
0,0,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,10,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Clay soil
1,230,230,230,229,229,230,229,229,201,198,199,230,230,231,231,231,231,232,222,185,219,230,229,229,229,229,229,229,222,170,159,173,150,147,154,148,146,151,148,145,...,210,216,208,219,208,210,192,199,206,211,213,205,206,189,211,210,194,143,106,118,129,146,188,211,209,194,210,206,217,211,209,202,221,219,198,204,202,196,192,Alluvial soil
2,141,140,160,175,144,156,142,126,133,156,175,165,136,131,140,134,127,135,121,119,120,128,176,163,143,138,152,144,141,134,121,122,132,139,138,129,103,86,89,111,...,102,103,108,111,110,108,104,102,101,102,101,108,128,102,109,119,114,112,124,119,104,109,100,106,120,129,146,136,121,119,142,149,144,126,98,110,108,112,127,Red soil
3,207,198,182,226,230,172,166,189,160,178,193,155,179,121,149,180,84,129,207,183,177,142,176,114,146,199,182,160,125,90,83,80,81,144,173,206,184,203,207,183,...,48,52,48,68,129,137,154,233,236,196,204,198,180,148,193,170,140,189,136,175,163,178,187,188,155,114,173,161,181,168,158,141,167,109,106,138,98,139,192,Clay soil
4,69,102,27,36,69,62,56,68,36,50,81,105,74,28,23,32,38,62,69,42,17,19,27,20,51,57,32,51,51,33,24,39,107,50,65,74,85,86,55,18,...,20,49,70,34,3,17,13,14,24,37,42,29,10,27,60,57,38,34,46,57,51,23,20,18,34,46,56,52,33,28,28,23,8,16,31,28,15,18,23,Black Soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,226,223,227,227,227,227,226,218,214,223,225,228,228,228,229,229,229,229,228,228,225,215,212,223,225,226,226,226,225,225,225,225,224,224,222,222,223,223,223,225,...,112,112,173,194,171,161,188,194,190,196,185,193,144,163,183,207,194,151,140,146,173,153,163,172,173,176,178,126,114,121,147,147,176,145,163,192,192,183,187,Alluvial soil
1210,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Clay soil
1211,117,142,117,71,79,40,49,134,105,47,125,117,108,104,72,107,89,33,101,120,44,44,95,125,113,77,69,48,28,81,94,111,77,94,83,97,122,133,106,142,...,26,16,22,11,10,27,26,4,64,108,122,60,23,21,17,26,26,10,13,11,15,9,20,4,16,42,4,23,18,78,103,67,40,20,37,41,16,12,13,Black Soil
1212,161,163,165,168,169,164,164,164,166,164,166,164,165,162,165,160,168,160,165,172,173,172,171,174,176,178,178,176,173,172,169,171,175,175,171,169,165,176,174,173,...,173,156,111,117,132,162,165,175,156,161,131,163,155,173,176,174,177,178,177,170,172,161,164,168,164,162,170,163,130,158,128,81,105,135,140,161,167,167,142,Alluvial soil


In [12]:
x,y = images_df.drop(columns=['soil_type']),images_df['soil_type']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
model = SVC(kernel='linear', C=1.0, random_state=42)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

Alluvial soil       0.92      0.95      0.93        96
   Black Soil       0.93      0.93      0.93        46
    Clay soil       0.94      0.71      0.81        41
     Red soil       0.85      0.95      0.90        60

     accuracy                           0.91       243
    macro avg       0.91      0.89      0.89       243
 weighted avg       0.91      0.91      0.90       243



In [15]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
model_scaled = SVC(kernel='linear', C=1.0, random_state=42)
model_scaled.fit(x_train_scaled, y_train)
y_pred_scaled = model_scaled.predict(x_test_scaled)
print(classification_report(y_test, y_pred_scaled))

               precision    recall  f1-score   support

Alluvial soil       0.91      0.96      0.93        96
   Black Soil       0.93      0.93      0.93        46
    Clay soil       1.00      0.71      0.83        41
     Red soil       0.85      0.95      0.90        60

     accuracy                           0.91       243
    macro avg       0.92      0.89      0.90       243
 weighted avg       0.92      0.91      0.91       243



In [21]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier

In [ ]:
forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
forest_model.fit(x_train, y_train)

y_pred_forest = forest_model.predict(x_test) 
print(classification_report(y_test, y_pred_forest))

               precision    recall  f1-score   support

Alluvial soil       0.93      0.95      0.94        96
   Black Soil       0.88      0.93      0.91        46
    Clay soil       1.00      0.80      0.89        41
     Red soil       0.86      0.90      0.88        60

     accuracy                           0.91       243
    macro avg       0.92      0.90      0.90       243
 weighted avg       0.91      0.91      0.91       243



In [32]:
forest_model2 = RandomForestClassifier(n_estimators=100, random_state=42)
forest_model2.fit(x_train_scaled, y_train)

y_pred_forest_sca = forest_model2.predict(x_test_scaled) 
print(classification_report(y_test, y_pred_forest_sca))

               precision    recall  f1-score   support

Alluvial soil       0.93      0.95      0.94        96
   Black Soil       0.88      0.93      0.91        46
    Clay soil       1.00      0.80      0.89        41
     Red soil       0.86      0.90      0.88        60

     accuracy                           0.91       243
    macro avg       0.92      0.90      0.90       243
 weighted avg       0.91      0.91      0.91       243



In [19]:
boost_model = AdaBoostClassifier(n_estimators=100, random_state=42)
boost_model.fit(x_train, y_train)
y_pred_boost = boost_model.predict(x_test)
print(classification_report(y_test, y_pred_boost))

               precision    recall  f1-score   support

Alluvial soil       0.73      0.74      0.74        96
   Black Soil       0.90      0.83      0.86        46
    Clay soil       0.56      0.49      0.52        41
     Red soil       0.63      0.72      0.67        60

     accuracy                           0.71       243
    macro avg       0.71      0.69      0.70       243
 weighted avg       0.71      0.71      0.71       243



In [ ]:
images_df["soil_type"].value_counts()

soil_type
Alluvial soil    525
Red soil         262
Black Soil       228
Clay soil        199
Name: count, dtype: int64

In [38]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(max_iter=1000)
logmodel.fit(x_train, y_train)
y_pred_log = logmodel.predict(x_test)
print(classification_report(y_test, y_pred_log))

               precision    recall  f1-score   support

Alluvial soil       0.90      0.94      0.92        96
   Black Soil       0.86      0.83      0.84        46
    Clay soil       0.88      0.73      0.80        41
     Red soil       0.83      0.90      0.86        60

     accuracy                           0.87       243
    macro avg       0.87      0.85      0.86       243
 weighted avg       0.87      0.87      0.87       243



In [37]:
logmodelsca = LogisticRegression(max_iter=1000)
logmodelsca.fit(x_train_scaled, y_train)
y_pred_logsca = logmodel.predict(x_test_scaled)
print(classification_report(y_test, y_pred_logsca))

               precision    recall  f1-score   support

Alluvial soil       0.91      0.80      0.85        96
   Black Soil       0.85      0.74      0.79        46
    Clay soil       0.57      0.76      0.65        41
     Red soil       0.75      0.80      0.77        60

     accuracy                           0.78       243
    macro avg       0.77      0.77      0.77       243
 weighted avg       0.80      0.78      0.79       243



C:\Users\A MUTHU SELVI ESWARI\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [40]:
y_pred_bagged = np.array([sorted(i,key=i.count, reverse=True)[0] for i in zip(y_pred, y_pred_scaled, y_pred_forest,y_pred_forest_sca, y_pred_log,y_pred_logsca)])
print(classification_report(y_test, y_pred_bagged))

               precision    recall  f1-score   support

Alluvial soil       0.93      0.95      0.94        96
   Black Soil       0.93      0.93      0.93        46
    Clay soil       0.94      0.76      0.84        41
     Red soil       0.86      0.95      0.90        60

     accuracy                           0.91       243
    macro avg       0.92      0.90      0.90       243
 weighted avg       0.92      0.91      0.91       243

